# Example: Cybernetic Portfolio Allocation Performance
Fill me in.

## Setup
We set up the computational environment by including [the `Include.jl` file](Include.jl). The `Include.jl` file loads external packages, various functions that we will use in the exercise, and custom types to model the components of our lab problem.
* For additional information on functions and types used in this material, see the [Julia programming language documentation](https://docs.julialang.org/en/v1/) and the [VLQuantitativeFinancePackage.jl documentation](https://github.com/varnerlab/VLQuantitativeFinancePackage.jl).

In [3]:
include("Include.jl");

## Prerequisites: Load and clean the testing dataset
We gathered a daily open-high-low-close `dataset` for each firm in the [S&P500](https://en.wikipedia.org/wiki/S%26P_500) from `01-03-2024` until last week `09-23-24` close, along with data for a few exchange-traded funds and volatility products during that time. We load the `out_of_sample_dataset::Dict{String, DataFrame}` by calling the `MyOutOfSampleMarketDataSet()` function:

In [5]:
out_of_sample_dataset = MyOutOfSampleMarketDataSet() |> x-> x["dataset"]

Dict{String, DataFrame} with 488 entries:
  "NI"   => 182×8 DataFrame…
  "EMR"  => 182×8 DataFrame…
  "CTAS" => 182×8 DataFrame…
  "HSIC" => 182×8 DataFrame…
  "KIM"  => 182×8 DataFrame…
  "PLD"  => 182×8 DataFrame…
  "IEX"  => 182×8 DataFrame…
  "BAC"  => 182×8 DataFrame…
  "CBOE" => 182×8 DataFrame…
  "EXR"  => 182×8 DataFrame…
  "NCLH" => 182×8 DataFrame…
  "CVS"  => 182×8 DataFrame…
  "DRI"  => 182×8 DataFrame…
  "DTE"  => 182×8 DataFrame…
  "ZION" => 182×8 DataFrame…
  "AVY"  => 182×8 DataFrame…
  "EW"   => 182×8 DataFrame…
  "EA"   => 182×8 DataFrame…
  "NWSA" => 182×8 DataFrame…
  "BBWI" => 182×8 DataFrame…
  "CAG"  => 182×8 DataFrame…
  "GPC"  => 182×8 DataFrame…
  "FCX"  => 182×8 DataFrame…
  "GILD" => 182×8 DataFrame…
  "UNP"  => 182×8 DataFrame…
  ⋮      => ⋮

In [6]:
out_of_sample_dataset["AAPL"];

### Load single index models and compute the covariance
Previously, we estimated the parameters and residual distributions for each `ticker` and saved these to a [JLD2.jl file](https://github.com/JuliaIO/JLD2.jl). Load this file using the [load(...) function](https://juliaio.github.io/JLD2.jl/stable/#save-and-load-functions). 
* Store the model collection in the `sim_model_dictionary` variable, where the keys of the dictionary are the ticker symbols and the values are the parameterized [MySingleIndexModel instances](https://varnerlab.github.io/VLQuantitativeFinancePackage.jl/dev/portfolio/#VLQuantitativeFinancePackage.MySingleIndexModel) estimated previously.

In [8]:
sim_model_dictionary = load(joinpath(_PATH_TO_DATA, "SIMs-SP500-01-03-18-to-12-29-23.jld2")) |> x->x["sim"]

Dict{String, MySingleIndexModel} with 460 entries:
  "NI"   => MySingleIndexModel(-0.0671224, 0.652999, 0.0389, Normal{Float64}(μ=…
  "EMR"  => MySingleIndexModel(-0.0549877, 1.23526, 0.0389, Normal{Float64}(μ=-…
  "CTAS" => MySingleIndexModel(0.119417, 1.17936, 0.0389, Normal{Float64}(μ=-6.…
  "HSIC" => MySingleIndexModel(-0.0811194, 0.913675, 0.0389, Normal{Float64}(μ=…
  "KIM"  => MySingleIndexModel(-0.0818621, 1.28859, 0.0389, Normal{Float64}(μ=-…
  "PLD"  => MySingleIndexModel(0.0273092, 1.02836, 0.0389, Normal{Float64}(μ=1.…
  "IEX"  => MySingleIndexModel(-0.00940007, 0.963259, 0.0389, Normal{Float64}(μ…
  "BAC"  => MySingleIndexModel(-0.0922207, 1.32778, 0.0389, Normal{Float64}(μ=-…
  "CBOE" => MySingleIndexModel(-0.00982405, 0.588498, 0.0389, Normal{Float64}(μ…
  "EXR"  => MySingleIndexModel(0.0252878, 0.725388, 0.0389, Normal{Float64}(μ=3…
  "NCLH" => MySingleIndexModel(-0.336719, 2.32891, 0.0389, Normal{Float64}(μ=1.…
  "CVS"  => MySingleIndexModel(-0.0703279, 0.78292, 0.0389

Next, let's build a sorted list of firms that we have in cleaned up `sim_model_dictionary::Dict{String, MySingleIndexModel}` and save it in the `list_of_all_tickers::Array{String,1}` array:

In [10]:
list_of_all_tickers = keys(sim_model_dictionary) |> collect |> sort;

Set some constants. Since we don't have access to historical data in this case study, we'll estimate some parameters from other sources or set typical values.

In [35]:
σₘ = 0.1621; # implied volatility for SPY
μₘ = 0.10; # assumed expected return for SPY
risk_free_rate = 0.0389; # hypothetical continuous compounded risk-free rate (units: 1/year)
Δt = (1.0/252.0); # time step for 1-trading day (units: 1/year)
initial_buffer_size = 10; 

#### Compute: Covariance matrix using single index models
Next, build the covariance array using single index models. Substituting the single index model for the excess return of asset $i$ and $j$ into the covariance expression (and simplifying) gives:
$$
\begin{equation*}
\text{cov}(R_{i}, R_{j}) = \begin{cases}
\beta_{i}^{2}\sigma_{m}^{2}+\sigma_{\epsilon_{i}}^{2} & i = j \\
\beta_{i}\beta_{j}\sigma_{m}^2 & i \neq j
\end{cases}
\end{equation*}
$$
where $\sigma_{m}^2$ denotes the variance of the excess return of the market,  and $\sigma_{\epsilon_{i}}^{2}$ denotes the variance of the firm-specific error model. We populate the single index covariance array `Σ` using nested [`for-loops.`](https://docs.julialang.org/en/v1/base/base/#for).

In [14]:
Σ = let
    Σ_tmp = Array{Float64,2}(undef, length(list_of_all_tickers), length(list_of_all_tickers));
    for i ∈ eachindex(list_of_all_tickers)
        
        outer_ticker = list_of_all_tickers[i]; # ticker on the rows 
        sim_outer = sim_model_dictionary[outer_ticker]; # get SIM model for row ticker
        
        for j ∈ eachindex(list_of_all_tickers)
            
            inner_ticker = list_of_all_tickers[j]; # ticker in the cols
            sim_inner = sim_model_dictionary[inner_ticker]; # get SIM model of the col ticker
            
            if (i == j) # we have a diagonal element
                βᵢ = sim_outer.β
                ϵᵢ = sim_outer.ϵ
                σ_ϵᵢ = params(ϵᵢ)[2];
                Σ_tmp[i,j] = ((βᵢ)^2)*((σₘ)^2)+(σ_ϵᵢ)^2
            else
                βᵢ = sim_outer.β
                βⱼ = sim_inner.β
                Σ_tmp[i,j] = βᵢ*βⱼ*(σₘ)^2
            end
        end
    end
    Σ_sim  = Σ_tmp |> x-> x*(1/252) # annualize
end;

## Task 1: Populate the 2024 Market Dataset and Form a Portfolio
Fill me in

In [16]:
my_list_of_tickers = ["AAPL", "MSFT", "MU", "AMD", "NVDA", "JNJ", "PG", "MRK", "PFE", "GS", "WFC", "JPM", "SPY"]; # your set of tickers + SPY

fill me in

In [32]:
market_df = let

    number_of_rows = nrow(out_of_sample_dataset["AAPL"]); # how many dates do we have in the out-of-sample dataset 
    market_df = DataFrame()
    for i ∈ 1:number_of_rows
        tmp = Dict{Symbol,Union{DateTime, Float64}}();
        for ticker ∈ my_list_of_tickers
            data_df = out_of_sample_dataset[ticker];
            tmp[Symbol(ticker)] = data_df[i,:volume_weighted_average_price];
        end
        push!(market_df, tmp |> NamedTuple);
    end
    market_df
end

Row,WFC,PG,MSFT,JPM,MRK,PFE,AAPL,NVDA,JNJ,GS,AMD,SPY,MU
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,48.6458,147.994,371.164,171.333,115.049,29.7929,184.323,47.6677,160.932,381.116,135.287,469.898,82.0679
2,49.4574,148.757,369.808,172.02,117.028,29.2061,182.018,48.0888,160.88,383.82,136.436,468.554,83.1953
3,50.0134,147.472,368.951,172.464,117.087,29.2798,181.474,49.0192,160.868,386.709,139.025,468.078,82.8071
4,49.5721,148.469,372.911,171.031,117.057,29.5119,184.37,51.3375,160.955,387.721,144.89,471.969,84.9118
5,49.2363,149.023,374.47,170.726,118.633,29.4901,184.371,53.1155,161.675,383.367,148.098,473.537,83.4795
6,48.9249,149.81,381.63,170.217,118.607,29.0134,185.251,54.1639,161.782,381.305,148.298,475.682,82.3212
7,48.8278,150.225,384.628,169.65,118.27,28.423,185.06,54.5998,161.177,378.449,146.862,475.315,82.8573
8,47.649,150.487,387.404,171.553,118.408,28.7143,185.82,54.6709,162.171,378.65,146.539,476.587,82.3265
9,46.7974,149.894,390.702,166.874,118.525,28.3475,182.887,56.1219,160.869,380.505,156.685,474.914,84.2181


Fill me in

## Task 2: Compute the 2024 performance of an equally weighted portfolio
Fill me in

## Task 3: Compute the 2024 performance of a cybernetic weighted portfolio
Fill me in.

## Disclaimer and Risks
__This content is offered solely for training and informational purposes__. No offer or solicitation to buy or sell securities or derivative products or any investment or trading advice or strategy is made, given, or endorsed by the teaching team. 

__Trading involves risk__. Carefully review your financial situation before investing in securities, futures contracts, options, or commodity interests. Past performance, whether actual or indicated by historical tests of strategies, is no guarantee of future performance or success. Trading is generally inappropriate for someone with limited resources, investment or trading experience, or a low-risk tolerance.  Only risk capital that is not required for living expenses.

__You are fully responsible for any investment or trading decisions you make__. You should decide solely based on your financial circumstances, investment or trading objectives, risk tolerance, and liquidity needs.